# Chapter 23 - Cross Validation

In [1]:
%run common.ipynb

Name: Error parsing magics!
Message: Magics [run] do not exist!
StackTrace: 

In [2]:
%ShowTypes on

Types will be printed.


# Setup

In [3]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row

import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame

import org.apache.spark.ml.feature._

import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

In [4]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<!-- To left align the HTML components in Markdown -->

### Spark parition control based on core availability

In [5]:
val NUM_CORES = 4
val NUM_PARTITIONS = 4

lazy val spark: SparkSession = SparkSession.builder()
    .appName("mllib-cross-validation")
    .getOrCreate()

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/
import spark.implicits._

NUM_CORES: Int = 4
NUM_PARTITIONS: Int = 4
spark: org.apache.spark.sql.SparkSession = <lazy>


spark: org.apache.spark.sql.SparkSession = <lazy>


In [6]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,192.168.0.27)
(spark.eventLog.enabled,true)
(spark.driver.port,37281)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://192.168.0.27:37281/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-58249d22-0765-49ec-bf06-75ba3402b330/repl-d197b9b6-0c22-4c41-a20d-0e2b6a7be661)
(spark.app.name,mllib-cross-validation)
(spark.driver.memory,3g)
(spark.executor.instances,2)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.default.parallelism,16)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://192.168.0.27:4040)
(spark.or

configMap: Unit = ()


## Constants

In [7]:
val PROTOCOL="file://"
val DATA_DIR="/home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data"
val RESULT_DIR="."

PROTOCOL: String = file://
DATA_DIR: String = /home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data
RESULT_DIR: String = .


RESULT_DIR: String = .


# Dataframe

In [8]:
var df = spark.read.json(PROTOCOL + DATA_DIR + "/simple-ml")

df: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]


df: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]


In [9]:
df.orderBy("value2").show(false)

+-----+----+------+------------------+
|color|lab |value1|value2            |
+-----+----+------+------------------+
|green|good|1     |14.386294994851129|
|green|bad |16    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|blue |bad |12    |14.386294994851129|
|green|bad |16    |14.386294994851129|
|green|good|12    |14.386294994851129|
|red  |good|35    |14.386294994851129|
|red  |good|35    |14.386294994851129|
|red  |bad |2     |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|green|good|1     |14.386294994851129|
|green|good|12    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|red  |good|35    |14.386294994851129|
|blue |bad |12    |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|green|good|12    |14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



# Split train/test data

In [10]:
val Array(train, test) = df.randomSplit(Array(0.7, 0.3))

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 2 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 2 more fields]


test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 2 more fields]


# Pipeline

## RFormula

In [11]:
val rFormula = new RFormula()

rFormula: org.apache.spark.ml.feature.RFormula = RFormula() (uid=rFormula_25eff8487eb7)


rFormula: org.apache.spark.ml.feature.RFormula = RFormula() (uid=rFormula_25eff8487eb7)


## Logistic regression

In [12]:
import org.apache.spark.ml.classification.LogisticRegression
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_e7c617bcc34d


lr: org.apache.spark.ml.classification.LogisticRegression = logreg_e7c617bcc34d


## Pipeline

In [13]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
val pipeline = new Pipeline()
    .setStages(Array(
        rFormula,
        lr
    ))

pipeline: org.apache.spark.ml.Pipeline = pipeline_23c2b6217066


pipeline: org.apache.spark.ml.Pipeline = pipeline_23c2b6217066


## Parameter Grid

In [14]:
import org.apache.spark.ml.tuning.ParamGridBuilder
val params = new ParamGridBuilder()
    .addGrid(
        rFormula.formula, 
        Array(
            "lab ~ . + color:value1",
            "lab ~ . + color:value1 + color:value2"
        )
    )
    .addGrid(
        lr.elasticNetParam, 
        Array(
            0.0, 0.5, 1.0
        )
    )
    .addGrid(
        lr.regParam, 
        Array(
            0.1, 2.0)
    )
    .build()

params: Array[org.apache.spark.ml.param.ParamMap] = 


Array({
	logreg_e7c617bcc34d-elasticNetParam: 0.0,
	rFormula_25eff8487eb7-formula: lab ~ . + color:value1,
	logreg_e7c617bcc34d-regParam: 0.1
}, {
	logreg_e7c617bcc34d-elasticNetParam: 0.5,
	rFormula_25eff8487eb7-formula: lab ~ . + color:value1,
	logreg_e7c617bcc34d-regParam: 0.1
}, {
	logreg_e7c617bcc34d-elasticNetParam: 1.0,
	rFormula_25eff8487eb7-formula: lab ~ . + color:value1,
	logreg_e7c617bcc34d-regParam: 0.1
}, {
	logreg_e7c617bcc34d-elasticNetParam: 0.0,
	rFormula_25eff8487eb7-formula: lab ~ . + color:value1,
	logreg_e7c617bcc34d-regParam: 2.0
}, {
	logreg_e7c617bcc34d-elasticNetParam: 0.5,
	rFormula_25eff8487eb7-formula: lab ~ . + color:value1,
	logreg_e7c617bcc34d-regParam: ...


params: Array[org.apache.spark.ml.param.ParamMap] = 


## Evaluator

In [15]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator = new BinaryClassificationEvaluator()
  .setMetricName("areaUnderROC")
  .setRawPredictionCol("prediction")
  .setLabelCol("label")

evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_cc4ce64dd640


evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_cc4ce64dd640


# Cross Validation

In [16]:
import org.apache.spark.ml.tuning.TrainValidationSplit
val tvs = new TrainValidationSplit()
  .setTrainRatio(0.75) // also the default.
  .setEstimatorParamMaps(params)
  .setEstimator(pipeline)
  .setEvaluator(evaluator)

tvs: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_9e03bf4c2bc8


tvs: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_9e03bf4c2bc8


In [17]:
val tvsFitted = tvs.fit(train)

tvsFitted: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_9e03bf4c2bc8


tvsFitted: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_9e03bf4c2bc8


In [18]:
evaluator.evaluate(tvsFitted.transform(test))

Double = 0.9


In [19]:
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.LogisticRegressionModel
val trainedPipeline = tvsFitted.bestModel.asInstanceOf[PipelineModel]
val TrainedLR = trainedPipeline.stages(1).asInstanceOf[LogisticRegressionModel]
val summaryLR = TrainedLR.summary
summaryLR.objectiveHistory

trainedPipeline: org.apache.spark.ml.PipelineModel = pipeline_23c2b6217066
TrainedLR: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_e7c617bcc34d, numClasses = 2, numFeatures = 7
summaryLR: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@1a0acc74


Array[Double] = Array(0.6834216514972252, 0.6788635376034742, 0.5013194529411205, 0.46886086049047837, 0.4582044702791959, 0.44381578912992786, 0.43694511597842656, 0.4340371570535083, 0.4310811790523375, 0.42692115429653704, 0.4244654215426553, 0.42389630615099694, 0.4238189128643...


# Persist model

In [20]:
tvsFitted.write.overwrite().save("/tmp/modelLocation")

## Reload and re-test

In [21]:
import org.apache.spark.ml.tuning.TrainValidationSplitModel
val model = TrainValidationSplitModel.load("/tmp/modelLocation")


model: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_9e03bf4c2bc8


model: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_9e03bf4c2bc8


In [22]:
model.transform(test).select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [23]:
evaluator.evaluate(model.transform(test))

Double = 0.9
